# CS170: Zoom Breakout Rooms
Input: n = # of students,  
Smax = Stress budget,  
h_i_j, s_i_j = happiness & stress between each student pair  

Output:  
H_r = Happiness of breakout room r  
S_r = stress of room r  
H_total = total happiness  
k = # of rooms opened  

Constraints:  
S_r < Smax / k  
k = # of rooms opened.   

Goal: maximize H total.

## importing files & libs

In [1]:
!ls

__pycache__
breakout-rooms.ipynb
input_create.ipynb
input_create.ipynb.txt
input_create.py
inputs
inputs_outputs
outputs
parse.py
README.md
reflection.txt
samples
solver.py
utils.py


In [157]:
import utils
import utils as ut
import parse as ps
import networkx as nx
from docplex.cp.model import *
from docplex.mp.model import Model
import numpy as np
import pandas

## Parsing Variables
Input: n = # of students,
Smax = Stress budget,
h_i_j, s_i_j = happiness & stress between each student pair

In [265]:
def process_input(filename):
    G, S_MAX = ps.read_input_file(filename)
    N = len(G)
    D = nx.to_dict_of_dicts(G)
    R = range(N)
    S_mat = np.zeros((N,N))
    H_mat = np.zeros((N,N))
    dic = D.copy()
    idx = []
    for i in R:
        dic.pop(i);
        idx.extend([(i, i, k) for k in R])
        for j in dic.keys():
            info = D[i][j]
            S_mat[i,j]=info['stress']
            H_mat[i,j]=info['happiness']
            idx.extend([(i, j, k) for k in R])
    #         D[i][j]['h'] = D[i][j].pop('happiness')
    #         D[i][j]['s'] = D[i][j].pop('stress')
    #         print("{}-{}: h={}, s={}".format(i, j, info['happiness'], info['stress']))
#     print(S_mat)
#     print(H_mat)
    print(len(idx))
    return G, S_MAX, N, R, S_mat, H_mat, idx

## Build Model (CP)

In [10]:
# Create model
mdl = CpoModel("breakout_rooms")
# Initialize model variable sets
# total_rooms = 0

In [62]:
def open_room(students_list):
    '''
    returns S_r, updates total_rooms, stress_of_rooms, and H_total
    '''
    S_r = 0
    for i in len(students_list):
        s = students_list.pop(0)
        for other_s in students_list:
            H_total += D[s][other_s]['h']
            S_r += D[s][other_s]['s']
    total_rooms += 1
    stress_of_rooms.append(S_r)
    return S_r

count_different()
sum()
mdl.add("s1 <= s2")

list

In [94]:
nbrooms = mdl.integer_var(0, N-1, "nbrooms") # number of rooms
S_room_max = S_MAX / nbrooms
room_assignment = mdl.integer_var_list(N, 0, N-1, "room_assignment")  # room_assignment[i]: the room student i is assigned to
# constraint nbrooms to be the same as the total number of rooms assigned
mdl.add(count_different(room_assignment) == nbrooms)
# students_in_rooms = [[] for i in range(N)]
# stress_of_rooms = []
# H_total = 0  # total happiness
# organize students by rooms they are assigned to
# for st, r in zip(range(N), room_assignment):
#     students_in_rooms[r].append(st)


# # constraint S_r (total stress in a room) < S_MAX/nbrooms
# for students in students_in_rooms:
#     l = students.copy()
#     S_r = 0
#     for i in len(l):
#         st = l.pop(0)
#         for other_st in l:
#             S_r += S_mat[st, other_st]
#             H_total += H_mat[st, other_st]
#     mdl.add(S_r < S_room_max)
    
    

for no_r in range(N): # for each room number
#     mdl.add(mdl.all_diff(x[i] - i for i in range(NB_QUEEN)))
    students_in_r = [st for st in range(N) if room_assignment[st] == no_r]
    st = students_in_r.pop(0)
    idx = [[],[]]
    for other_st in students_in_r:
        idx[0].append(st)
        idx[1].append(other_st)
        mdl.add(sum(S_mat[idx]) < S_room_max)
# constraint maximize total happiness
mdl.add(mdl.maximize(H_total))
msol = mdl.solve(TimeLimit=20, LogPeriod=3000)
msol

CpoException: CPO expression can not be used as boolean.


# Build model (MP)

In [266]:
G, S_MAX, N, R, S_mat, H_mat, idx = process_input("inputs_outputs/50.in")

63750


In [241]:
# ref to the mp sudoku example: https://ibmdecisionoptimization.github.io/docplex-doc/mp/creating_model.html
model = Model("breakout_room_mp")
R = range(N)
# X is room assignment; key is (i, j, r) where i,j is a pair of students, and r is the room they are assigned to
x = model.binary_var_dict(idx, name="X") 
# room_count = model.integer_var(1, N, "room_count")

# constraint 1: each pair can only be assigned to one room. 
# -> for each pair, sum of all different room choices == 1
# constaint 1.1: everyone can only be assigned to one room.
# -> sum of one students assignment == 1
for i in R:
#     model.add_constraint(model.sum(x[j,i,k] for j in range(i) for k in R)
#                     +    model.sum(x[i,j,k] for j in range(i,N) for k in R) == 1)
    for j in range(i, N):
        pair_sum = model.sum(x[i,j,k] for k in R)
        model.add_constraint(pair_sum == 1)

# constraint 1.2
for k in range(N):
    for i in R:
        for j in range(i, N):
            model.add_constraint(model.if_then(x[i,j,k], x[i,j,k+1]==0))

# constraint 2: rooms are using consecutive numbers starting from 0. 
# if nobody is in one room, do not assign people to rooms with no > this one
# also: calculate room count
# now: count all partners in the assigned group!
for k in range(N-1):
#     room_sum = model.sum(x[i,j,k] for i in R for j in range(i,N))
    for i in R:
        for j in range(i, N):
#             model.add_constraint(model.if_then(room_sum == 0, x[i,j,k+1]==0))
#             model.add_constraint(model.if_then(room_sum == 0, room_count <= k))
#             model.add_constraint(model.if_then(room_sum != 0, room_count >= k + 1))

# constraint 3: stress level of each room is under the limit. 
# how to find the number of rooms assigned? <- create a new variable
for r in R:
    S_room = model.sum(S_mat[i,j] * x[i,j,r] for i in R for j in range(i,N))
    model.add_constraint(S_room <= S_MAX / 5)
#     model.add_constraint(S_room * room_count <= S_MAX)
# goal: maximize happiness
model.maximize(model.sum(H_mat[i,j] * x[i,j,k] for i in R for j in range(i,N) for k in R))

In [242]:
model.set_time_limit(10)
msol = model.solve()

In [243]:
print(msol)

solution for: breakout_room_mp
objective: 2425.33
X_0_38_24=1
X_1_21_0=1
X_2_40_0=1
X_3_36_1=1
X_4_31_1=1
X_5_32_1=1
X_6_13_0=1
X_7_19_18=1
X_8_25_0=1
X_9_47_0=1
X_10_26_1=1
X_11_33_0=1
X_12_45_0=1
X_14_15_0=1
X_16_18_0=1
X_17_23_0=1
X_20_49_0=1
X_22_37_0=1
X_24_42_0=1
X_27_48_2=1
X_28_43_0=1
X_29_41_2=1
X_30_46_1=1
X_34_39_1=1
X_35_44_1=1



In [244]:
msol_dict = msol.get_value_dict(x)
msol_np = np.concatenate((list(msol_dict.keys()), np.array([list(msol_dict.values())]).T), axis=1).astype(int)
msol_nz = msol_np[msol_np[:,3].nonzero()]
msol_nz

array([[ 0, 38, 24,  1],
       [ 1, 21,  0,  1],
       [ 2, 40,  0,  1],
       [ 3, 36,  1,  1],
       [ 4, 31,  1,  1],
       [ 5, 32,  1,  1],
       [ 6, 13,  0,  1],
       [ 7, 19, 18,  1],
       [ 8, 25,  0,  1],
       [ 9, 47,  0,  1],
       [10, 26,  1,  1],
       [11, 33,  0,  1],
       [12, 45,  0,  1],
       [14, 15,  0,  1],
       [16, 18,  0,  1],
       [17, 23,  0,  1],
       [20, 49,  0,  1],
       [22, 37,  0,  1],
       [24, 42,  0,  1],
       [27, 48,  2,  1],
       [28, 43,  0,  1],
       [29, 41,  2,  1],
       [30, 46,  1,  1],
       [34, 39,  1,  1],
       [35, 44,  1,  1]])

In [245]:
msol_room_0 = msol_nz[np.argwhere(msol_nz[:,2] == 0).flatten()]
msol_room_0

array([[ 1, 21,  0,  1],
       [ 2, 40,  0,  1],
       [ 6, 13,  0,  1],
       [ 8, 25,  0,  1],
       [ 9, 47,  0,  1],
       [11, 33,  0,  1],
       [12, 45,  0,  1],
       [14, 15,  0,  1],
       [16, 18,  0,  1],
       [17, 23,  0,  1],
       [20, 49,  0,  1],
       [22, 37,  0,  1],
       [24, 42,  0,  1],
       [28, 43,  0,  1]])

In [246]:
msol_room_0[:,0]

array([ 1,  2,  6,  8,  9, 11, 12, 14, 16, 17, 20, 22, 24, 28])

In [247]:
S_mat[msol_room_0[:,0], msol_room_0[:,1]].sum()

18.67

In [248]:
S_MAX / 5

19.368199999999998

In [254]:
msol_rooms = msol_nz[:,2]
msol_unique_rooms = np.unique(msol_rooms)
msol_unique_rooms
number_of_rooms = len(msol_unique_rooms)

In [262]:
np.where(msol_rooms==msol_unique_rooms[i],i,msol_rooms)

array([4, 0, 0, 1, 1, 1, 0, 3, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2,
       1, 1, 1])

In [263]:
for i in range(number_of_rooms):
    msol_rooms = np.where(msol_rooms==msol_unique_rooms[i],i,msol_rooms)
msol_rooms
msol_nz[:,2]=msol_rooms
msol_nz

array([[ 0, 38,  4,  1],
       [ 1, 21,  0,  1],
       [ 2, 40,  0,  1],
       [ 3, 36,  1,  1],
       [ 4, 31,  1,  1],
       [ 5, 32,  1,  1],
       [ 6, 13,  0,  1],
       [ 7, 19,  3,  1],
       [ 8, 25,  0,  1],
       [ 9, 47,  0,  1],
       [10, 26,  1,  1],
       [11, 33,  0,  1],
       [12, 45,  0,  1],
       [14, 15,  0,  1],
       [16, 18,  0,  1],
       [17, 23,  0,  1],
       [20, 49,  0,  1],
       [22, 37,  0,  1],
       [24, 42,  0,  1],
       [27, 48,  2,  1],
       [28, 43,  0,  1],
       [29, 41,  2,  1],
       [30, 46,  1,  1],
       [34, 39,  1,  1],
       [35, 44,  1,  1]])

In [264]:
sol_dict = {}
for i in range(number_of_rooms):
    sel = msol_nz[np.argwhere(msol_rooms == i).flatten()]
    sol_dict.update(dict(zip(sel[:,0], sel[:,2])))
    sol_dict.update(dict(zip(sel[:,1], sel[:,2])))
sol_dict

{1: 0,
 2: 0,
 6: 0,
 8: 0,
 9: 0,
 11: 0,
 12: 0,
 14: 0,
 16: 0,
 17: 0,
 20: 0,
 22: 0,
 24: 0,
 28: 0,
 21: 0,
 40: 0,
 13: 0,
 25: 0,
 47: 0,
 33: 0,
 45: 0,
 15: 0,
 18: 0,
 23: 0,
 49: 0,
 37: 0,
 42: 0,
 43: 0,
 3: 1,
 4: 1,
 5: 1,
 10: 1,
 30: 1,
 34: 1,
 35: 1,
 36: 1,
 31: 1,
 32: 1,
 26: 1,
 46: 1,
 39: 1,
 44: 1,
 27: 2,
 29: 2,
 48: 2,
 41: 2,
 7: 3,
 19: 3,
 0: 4,
 38: 4}

In [267]:
ut.is_valid_solution(sol_dict,G,S_MAX, 5)

too stressful!  814.107


False